<a href="https://colab.research.google.com/github/umniahameed/DeepLearningHW/blob/main/DLHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import libraries and load data
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Add
import tensorflow as tf

# Load UCI Covertype dataset
data = fetch_covtype()
X = data.data
y = data.target - 1  # Convert to 0–6

# Normalize feature
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Onehot encod labels
y_encoded = to_categorical(y)

# Split into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# One batch of 128 samples for overfitting
X_overfit = X_train[:128]
y_overfit = y_train[:128]

input_shape = X_train.shape[1]
num_classes = y_encoded.shape[1]


In [ ]:
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model

inputs = Input(shape=(input_shape,), name="Input")

# Initial Dense layer
x = Dense(64, activation='relu', name="InitialDense")(inputs)

# Residual block
res_input = x
res = Dense(64, activation='relu', name="ResDense1")(res_input)
res = Dense(64, activation='relu', name="ResDense2")(res)
res_output = Add(name="ResidualAdd")([res_input, res])

skip = Dense(64, activation='relu', name="SkipConnection")(inputs)
merged = Add(name="FinalAdd")([res_output, skip])

# Final layr before output
final = Dense(32, activation='relu', name="FinalDense")(merged)

# Outpyt layer
outputs = Dense(num_classes, activation='softmax', name="Output")(final)

# Build model
model = Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)        │ (None, 54)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ InitialDense (Dense)      │ (None, 64)             │          3,520 │ Input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ResDense1 (Dense)         │ (None, 64)             │          4,160 │ InitialDense[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ResDense2 (Dense)         │ (None, 64)             │          4,160 │ ResDense1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ResidualAdd (Add)         │ (None, 64)             │              0 │ InitialDense[0][0],    │
│                           │                        │                │ ResDense2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ SkipConnection (Dense)    │ (None, 64)             │          3,520 │ Input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FinalAdd (Add)            │ (None, 64)             │              0 │ ResidualAdd[0][0],     │
│                           │                        │                │ SkipConnection[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FinalDense (Dense)        │ (None, 32)             │          2,080 │ FinalAdd[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Output (Dense)            │ (None, 7)              │            231 │ FinalDense[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,671 (69.03 KB)

 Trainable params: 17,671 (69.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train only on 128 samples
history = model.fit(X_overfit, y_overfit, epochs=500, verbose=0)

# Evaluate on full validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)

# Printung final results
print("Number of parameters:", model.count_params())
print("Final training loss:", history.history['loss'][-1])
print("Final validation loss:", val_loss)


Number of parameters: 17671
Final training loss: 9.219283674610779e-05
Final validation loss: 4.9283013343811035


In [ ]:
# Save model
model.save("mlp_res_skip.h5")
